In [ ]:
import time

import matplotlib.pyplot as plt
import numpy as np
import math

import cirq
from cirq.circuits.qasm_output import QasmUGate
import qsimcirq
import sympy as sp

import cupy as cp
from cuquantum import custatevec as cusv

from scipy.optimize import minimize

In [ ]:
#Qubits on Square-Octagon Lattice w/ Lattice Parameters
qubits = [0, 1, 2, 3, 4, 5, 6, 7]
Nq = len(qubits)

xx_links = [(0, 1), (2, 3)]
yy_links = [(0, 3), (1, 2)]
zz_links = [(0, 4), (1, 5), (2, 6), (3, 7)]

Jx, Jy, Jz = -1.0/math.sqrt(2), -1.0/math.sqrt(2), -1.0
hX = hY = hZ = 0.05/math.sqrt(3)

In [ ]:
#Hamiltonian Construction

#Qubit register for Hamiltonian
qreg = cirq.NamedQubit.range(Nq,prefix="q")

#Constructing Pauli Strings
XX_links_strings = []
for (i,j) in xx_links:
    XX_string = []
    for q in range(Nq):
        if q != i and q != j:
            XX_string.append(cirq.I(qreg[q]))
        else:
            XX_string.append(cirq.X(qreg[q]))
    XX_links_strings.append(XX_string)
    
YY_links_strings = []
for (i,j) in yy_links:
    YY_string = []
    for q in range(Nq):
        if q != i and q != j:
            YY_string.append(cirq.I(qreg[q]))
        else:
            YY_string.append(cirq.Y(qreg[q]))
    YY_links_strings.append(YY_string)

ZZ_links_strings = []
for (i,j) in zz_links:
    ZZ_string = []
    for q in range(Nq):
        if q != i and q != j:
            ZZ_string.append(cirq.I(qreg[q]))
        else:
            ZZ_string.append(cirq.Z(qreg[q]))
    ZZ_links_strings.append(ZZ_string)

X_qubit_strings = []
Y_qubit_strings = []
Z_qubit_strings = []
for k in qubits:
    X_string = []
    Y_string = []
    Z_string = []
    for q in range(Nq):
        if q != k:
            X_string.append(cirq.I(qreg[q]))
            Y_string.append(cirq.I(qreg[q]))
            Z_string.append(cirq.I(qreg[q]))
        else:
            X_string.append(cirq.X(qreg[q]))
            Y_string.append(cirq.Y(qreg[q]))
            Z_string.append(cirq.Z(qreg[q]))
    X_qubit_strings.append(X_string)
    Y_qubit_strings.append(Y_string)
    Z_qubit_strings.append(Z_string)

In [ ]:
#Hamiltonian Construction (ctd.)

#Convert Pauli Strings into Products
XX_links_terms = []
for XX_link in XX_links_strings:
    XX_links_terms.append(cirq.PauliString(XX_link))

YY_links_terms = []
for YY_link in YY_links_strings:
    YY_links_terms.append(cirq.PauliString(YY_link))
    
ZZ_links_terms = []
for ZZ_link in ZZ_links_strings:
    ZZ_links_terms.append(cirq.PauliString(ZZ_link))

X_qubit_terms = []
for qubit_string in X_qubit_strings:
    X_qubit_terms.append(cirq.PauliString(qubit_string))

Y_qubit_terms = []
for qubit_string in Y_qubit_strings:
    Y_qubit_terms.append(cirq.PauliString(qubit_string))

Z_qubit_terms = []
for qubit_string in Z_qubit_strings:
    Z_qubit_terms.append(cirq.PauliString(qubit_string))

In [ ]:
#Hamiltonian Construction (ctd.)

#Add up all the terms
Hamiltonian = 0
for XX_link in XX_links_terms:
    Hamiltonian += Jx*XX_link
for YY_link in YY_links_terms:
    Hamiltonian += Jy*YY_link
for ZZ_link in ZZ_links_terms:
    Hamiltonian += Jz*ZZ_link
for X_qubit in X_qubit_terms:
    Hamiltonian += hX*X_qubit
for Y_qubit in Y_qubit_terms:
    Hamiltonian += hY*Y_qubit
for Z_qubit in Z_qubit_terms:
    Hamiltonian += hZ*Z_qubit

In [ ]:
##Hardware Efficient Ansatz 2 Construction
def HEA_2_circuit(n_layers):
    global theta_0
    global theta_1
    global theta_2
    global theta_3
    global theta_4
    global theta_5
    global theta_6
    global theta_7
    global theta_8
    global theta_9
    global theta_10
    global theta_11
    global theta_12
    global theta_13
    global theta_14
    global theta_15
    global theta_16
    global theta_17
    global theta_18
    global theta_19
    global theta_20
    global theta_21
    global theta_22
    global theta_23
    #Initialize Circuit
    HEA_2 = cirq.Circuit()
    qreg_HEA_2 = cirq.NamedQubit.range(Nq, prefix="q")
    #Declare variational parameters
    theta_0 = sp.symarray('theta_0', n_layers)
    theta_1 = sp.symarray('theta_1', n_layers)
    theta_2 = sp.symarray('theta_2', n_layers)
    theta_3 = sp.symarray('theta_3', n_layers)
    theta_4 = sp.symarray('theta_4', n_layers)
    theta_5 = sp.symarray('theta_5', n_layers)
    theta_6 = sp.symarray('theta_6', n_layers)
    theta_7 = sp.symarray('theta_7', n_layers)
    theta_8 = sp.symarray('theta_8', n_layers)
    theta_9 = sp.symarray('theta_9', n_layers)
    theta_10 = sp.symarray('theta_10', n_layers)
    theta_11 = sp.symarray('theta_11', n_layers)
    theta_12 = sp.symarray('theta_12', n_layers)
    theta_13 = sp.symarray('theta_13', n_layers)
    theta_14 = sp.symarray('theta_14', n_layers)
    theta_15 = sp.symarray('theta_15', n_layers)
    theta_16 = sp.symarray('theta_16', n_layers)
    theta_17 = sp.symarray('theta_17', n_layers)
    theta_18 = sp.symarray('theta_18', n_layers)
    theta_19 = sp.symarray('theta_19', n_layers)
    theta_20 = sp.symarray('theta_20', n_layers)
    theta_21 = sp.symarray('theta_21', n_layers)
    theta_22 = sp.symarray('theta_22', n_layers)
    theta_23 = sp.symarray('theta_23', n_layers)
    #Ansatz Construction
    for idx_layer in range(n_layers):
        HEA_2.append(QasmUGate(theta_0[idx_layer]/np.pi, theta_1[idx_layer]/np.pi, theta_2[idx_layer]/np.pi)(qreg_HEA_2[0]))
        HEA_2.append(QasmUGate(theta_3[idx_layer]/np.pi, theta_4[idx_layer]/np.pi, theta_5[idx_layer]/np.pi)(qreg_HEA_2[1]))
        HEA_2.append(QasmUGate(theta_6[idx_layer]/np.pi, theta_7[idx_layer]/np.pi, theta_8[idx_layer]/np.pi)(qreg_HEA_2[2]))
        HEA_2.append(QasmUGate(theta_9[idx_layer]/np.pi, theta_10[idx_layer]/np.pi, theta_11[idx_layer]/np.pi)(qreg_HEA_2[3]))
        HEA_2.append(QasmUGate(theta_12[idx_layer]/np.pi, theta_13[idx_layer]/np.pi, theta_14[idx_layer]/np.pi)(qreg_HEA_2[4]))
        HEA_2.append(QasmUGate(theta_15[idx_layer]/np.pi, theta_16[idx_layer]/np.pi, theta_17[idx_layer]/np.pi)(qreg_HEA_2[5]))
        HEA_2.append(QasmUGate(theta_18[idx_layer]/np.pi, theta_19[idx_layer]/np.pi, theta_20[idx_layer]/np.pi)(qreg_HEA_2[6]))
        HEA_2.append(QasmUGate(theta_21[idx_layer]/np.pi, theta_22[idx_layer]/np.pi, theta_23[idx_layer]/np.pi)(qreg_HEA_2[7]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[0],qreg_HEA_2[1]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[0],qreg_HEA_2[2]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[0],qreg_HEA_2[3]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[0],qreg_HEA_2[4]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[0],qreg_HEA_2[5]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[0],qreg_HEA_2[6]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[0],qreg_HEA_2[7]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[1],qreg_HEA_2[2]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[1],qreg_HEA_2[3]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[1],qreg_HEA_2[4]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[1],qreg_HEA_2[5]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[1],qreg_HEA_2[6]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[1],qreg_HEA_2[7]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[2],qreg_HEA_2[3]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[2],qreg_HEA_2[4]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[2],qreg_HEA_2[5]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[2],qreg_HEA_2[6]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[2],qreg_HEA_2[7]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[3],qreg_HEA_2[4]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[3],qreg_HEA_2[5]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[3],qreg_HEA_2[6]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[3],qreg_HEA_2[7]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[4],qreg_HEA_2[5]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[4],qreg_HEA_2[6]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[4],qreg_HEA_2[7]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[5],qreg_HEA_2[6]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[5],qreg_HEA_2[7]))
        HEA_2.append(cirq.CNOT(qreg_HEA_2[6],qreg_HEA_2[7]))
    return HEA_2

In [ ]:
##Construct Objective Function
def objective(params, hamiltonian, qreg, simulator):
    ansatz = HEA_2_circuit(n_layers)
    params_dict = {'theta_0_0': params[0], 'theta_1_0': params[1], 'theta_2_0': params[2], 'theta_3_0': params[3], 'theta_4_0': params[4], 'theta_5_0': params[5], 'theta_6_0': params[6], 'theta_7_0': params[7], 'theta_8_0': params[8], 'theta_9_0': params[9], 'theta_10_0': params[10], 'theta_11_0': params[11], 'theta_12_0': params[12], 'theta_13_0': params[13], 'theta_14_0': params[14], 'theta_15_0': params[15], 'theta_16_0': params[16], 'theta_17_0': params[17], 'theta_18_0': params[18], 'theta_19_0': params[19], 'theta_20_0': params[20], 'theta_21_0': params[21], 'theta_22_0': params[22], 'theta_23_0': params[23]}
    expect = simulator.simulate_expectation_values(ansatz, hamiltonian, params_dict)
    return np.real(expect)

In [ ]:
##Perform VQE

#Initialize the simulator
gpu_options = qsimcirq.QSimOptions(use_gpu=True, gpu_mode=1)
simulator = qsimcirq.QSimSimulator(qsim_options=gpu_options)
qreg = cirq.NamedQubit.range(Nq, prefix="q")

#Specify number of layers
n_layers = 1

#Specify number of runs
n_runs = 101

#Loop that executes VQE
start_time = time.time()
optimal_values = []
cost_function_evals_collection = []
initial_values_collection = []
optimal_parameters_collection = []
for run in range(n_runs):
    x_init = np.random.uniform(0, 2*np.pi, size=24*n_layers)
    res = minimize(objective, x0=x_init, args=(Hamiltonian, qreg, simulator), method='COBYLA')
    optimal_values.append(res.fun)
    cost_function_evals_collection.append(res.nfev)
    initial_values_collection.append(x_init)
    optimal_parameters_collection.append(res.x)
end_time = time.time()
print(f"n_layers: {n_layers}")
print(f"Optimal Value: {min(optimal_values)}")
print(f"Cost Function Evaluations: {cost_function_evals_collection[np.argmin(optimal_values)]}")
print(f"Elapsed Time: {end_time - start_time}")
print("*" * 40)

In [ ]:
np.savetxt('Data for Diana (HEA 2 Results)/8_Qubit_1_Layer_HEA_2_Initial_Points_GPU.csv', initial_values_collection)
np.savetxt('Data for Diana (HEA 2 Results)/8_Qubit_1_Layer_HEA_2_Optimal_Values_GPU.csv', optimal_values)
np.savetxt('Data for Diana (HEA 2 Results)/8_Qubit_1_Layer_HEA_2_Optimal_Parameters_GPU.csv', optimal_parameters_collection)
np.savetxt('Data for Diana (HEA 2 Results)/8_Qubit_1_Layer_HEA_2_Cost_Function_Evaluations_GPU.csv', cost_function_evals_collection)